## Actividad: Uso de claves asimétricas

En esta actividad se comparte un mensaje cifrado por medio de una clave asimétrica. Si bien en este cuaderno se hace la codificación y la decodificación, es necesario que los estudiantes compartan los mensajes y la clave pública para que entiendan la dinámica de funcionamiento del par de claves privada y pública

Se recomienda que trabajen en grupos que compartirán los mensajes y las claves públicas


In [1]:
# Importar las bibliotecas necesarias
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes


In [8]:
# Generar un par de claves (privada y pública)
def generar_claves_asimetricas():
    """Genera y guarda un par de claves asimétricas en archivos."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()

    # Guardar clave privada
    with open("clave_privada.pem", "wb") as clave_privada_archivo:
        clave_privada_archivo.write(
            clave_privada.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.TraditionalOpenSSL,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    # Guardar clave pública
    with open("clave_publica.pem", "wb") as clave_publica_archivo:
        clave_publica_archivo.write(
            clave_publica.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

    print("Claves asimétricas generadas y guardadas en 'clave_privada.pem' y 'clave_publica.pem'.")

# Cargar clave privada
def cargar_clave_privada():
    """Carga la clave privada desde un archivo."""
    with open("clave_privada.pem", "rb") as clave_privada_archivo:
        return serialization.load_pem_private_key(
            clave_privada_archivo.read(),
            password=None
        )

# Cargar clave pública
def cargar_clave_publica():
    """Carga la clave pública desde un archivo."""
    with open("clave_publica.pem", "rb") as clave_publica_archivo:
        return serialization.load_pem_public_key(
            clave_publica_archivo.read()
        )

# Cifrar un mensaje asimétrico
def cifrar_mensaje_asimetrico(mensaje, clave_publica):
    """Cifra un mensaje utilizando la clave pública."""
    mensaje_cifrado = clave_publica.encrypt(
        mensaje.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_cifrado

# Descifrar un mensaje asimétrico
def descifrar_mensaje_asimetrico(mensaje_cifrado, clave_privada):
    """Descifra un mensaje cifrado utilizando la clave privada."""
    mensaje_descifrado = clave_privada.decrypt(
        mensaje_cifrado,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return mensaje_descifrado.decode()

# Validar la autenticidad del remitente
def validar_autenticidad(mensaje, firma, clave_publica):
    """Valida que el mensaje proviene del remitente verificando la firma con la clave pública."""
    try:
        clave_publica.verify(
            firma,
            mensaje.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception:
        return False

# Firmar un mensaje
def firmar_mensaje(mensaje, clave_privada):
    """Genera una firma para el mensaje utilizando la clave privada."""
    firma = clave_privada.sign(
        mensaje.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return firma


mensaje = "Una máquina simple es un dispositivo mecánico que cambia la dirección o la magnitud de una fuerza"

generar_claves_asimetricas()
clave_publica = cargar_clave_publica()
clave_privada = cargar_clave_privada()
mensaje_cifrado_asimetrico = cifrar_mensaje_asimetrico(mensaje, clave_publica)
print(f"Mensaje cifrado (asimétrico): {mensaje_cifrado_asimetrico}")
mensaje_descifrado_asimetrico = descifrar_mensaje_asimetrico(mensaje_cifrado_asimetrico, clave_privada)
print(f"Mensaje descifrado (asimétrico): {mensaje_descifrado_asimetrico}")


In [13]:

# Parte 3: Validación de Autenticidad firmando con la clave privada
print("\n[Validación de Autenticidad]")
firma = firmar_mensaje(mensaje, clave_privada)
print(f"Firma generada: {firma}")
es_valido = validar_autenticidad(mensaje, firma, clave_publica)
print(f"¿El mensaje es auténtico? {'Sí' if es_valido else 'No'}")



[Validación de Autenticidad]
Firma generada: b'%\xe9\xad\xbd7\xa4\x0e\xe7\xc8B#c9\xc1\xca\xf1c\x03\xae\xf6T\xd9]\xbcq\xfe\xaf\xf0#b\xf6\x1c\xd1{z6\x9b\xbe\x9a\xc9B\x9c\xb9\xa4\x8eq\x88\x1d\xa07:3\xc7\x13:2\xd6\xf4\xef\xa7r\x14\xe7\xf3\x87\x1d\xad\xc1Q\\\x0f\xd3\xf5\x8a\x8aY$\x90\x15\x84\xfa\xdb3j\x8d\x8e\xa5\xd7eq\x1d\xff\x9b\x13F\xb64\x8c\xd0u\x0b\x9eL!\x85e\x9e~j^\xb5\x9e\x92\x0e\xe9\xe9\xb8\xe0d=\x82\x9fN\t\x98-\xfe\x13\xd66\xc3\x88ZS\x0f\xd5v\xd9\x0f\x17|\xbf\xf4\xf1B)\x02\x15\xcdk\xf9\x82;nS\xf8\xee\x91\xb2)W\x92\x05*q\xe7\x00!&0\xf6\x958\x05\xd2A\xe5T=\x1c\x80\xeb\x92 5\xaa!\xce\xc0\xfd\xcf\x83\x04\x8b\xbb\x9f\x97U\x91\xac\x03\rt*\x1e\xd6\xaa\t\x80\xee\xb4\\\x11v\xf7q.!\xaf\xd8\x80n\x00\xe7<\x12"C\x81%hx5[\x9a\x7f\xda\x96Z\xc5\x0e\xa3`\xdf \xd0w\xe34\xfd\xcf\x1d\x02\xbe\x83j'
¿El mensaje es auténtico? Sí


### Actividad de reflexión
Discutir con los estudiantes:
- ¿Qué ventajas tiene el cifrado asimétrico?
- ¿Qué posibles fallas de seguridad pueden existir?
- ¿Qué sucede si yo pierdo mi clave privada? ¿Qué sucede si la comparto?
- Intente codificar la fábula de la lechera u otro texto largo ¿Por qué da un error? ¿Cómo se puede arreglar este error para compartir mensajes cifrados?
- Comparta mensajes largos cifrados entre los estudiantes y que ellos validen la autencididad del remitente